In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import random
import time

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc, roc_auc_score as auc
from sklearn import svm

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dot, Embedding, Flatten, Dense
from sklearn.cluster import KMeans

from sklearn.kernel_approximation import PolynomialCountSketch

In [3]:
def read_node_labels(filename_nodes_to_graph, filename_node_labels, nr_graphs):
    
    with open(filename_nodes_to_graph) as f_nodes:
        nodes = f_nodes.read().splitlines()
        
    with open(filename_node_labels) as f_labels:
        labels = f_labels.read().splitlines()
    
    if (len(nodes) != len(labels)):
        raise ValueError('Node lists of different length')
        return -1
    
    Vs = [{} for _ in range(nr_graphs)]
    nodes_to_graph = {}
    node_labels = {}
    set_labels = set()
    for i in range(len(nodes)):
        node_id = i+1
        graph_id = int(nodes[i])
        nodes_to_graph[node_id] = graph_id
        label = labels[i]
        set_labels.add(label)
        node_labels[node_id] = label
        Vs[graph_id][node_id] = label
    return Vs, nodes_to_graph, node_labels, set_labels

In [4]:
def read_edges(filename_edges, Vs, nodes_to_graph, node_labels, nr_graphs, sep=','):
    Es = [{} for _ in range(nr_graphs)]
    f_edges = open(filename_edges, 'r')
    for line in f_edges: 
        line_split = str.split(line, sep)
        e1 = int(line_split[0].strip())
        e2 = int(line_split[1].strip())
        if (nodes_to_graph[e1] != nodes_to_graph[e2]):
            print('Vertices connected by and edge but belonging to different graphs')
            print('nodes',  e1, e2)
            print('graphs', nodes_to_graph[e1], nodes_to_graph[e2])
        E = Es[nodes_to_graph[e1]]  
        L = []
        if e1 in E:
            L = E[e1]
        L.append(e2)
        E[e1] = L
        #Es[nodes_to_graph[e1]] = E
    return Es

In [5]:
def read_graph(folderpath, filename):
    '''
    A method for reading graphs in the format provided in 
    https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets
    '''
    
    filename_edges = folderpath + '/' + filename + '/' + filename + '_A.txt'
    filename_nodes_to_graph = folderpath + '/' + filename + '/' + filename + '_graph_indicator.txt'
    filename_node_labels = folderpath + '/' + filename + '/' + filename + '_node_labels.txt'
    filename_classes = folderpath + '/' + filename + '/' + filename + '_graph_labels.txt'
    
    print(filename_edges)
    
    classes = []
    with open(filename_classes) as f_classes:
        classes_f = f_classes.read().splitlines()  
        for c in classes_f:
            classes.append(int(c))
        
    nr_graphs = len(classes) + 1
    Vs, nodes_to_graph, node_labels, set_labels = read_node_labels(filename_nodes_to_graph, filename_node_labels, nr_graphs)
    Es = read_edges(filename_edges, Vs, nodes_to_graph, node_labels, nr_graphs)
    
    return Vs[1:], Es[1:], node_labels, classes, set_labels

In [15]:
path = "/home/koki/Desktop/Data/Graphs"
filename = "AIDS"
Vs, Es, node_labels, classes, set_labels = read_graph(path, filename)

/home/koki/Desktop/Data/Graphs/AIDS/AIDS_A.txt


In [16]:
len(Vs), len(Es), len(classes), len(set_labels)

(2000, 2000, 2000, 38)

In [17]:
np.mean(classes), min(classes), max(classes)

(0.8, 0, 1)

In [18]:
len(node_labels)

31385

In [19]:
def create_nx_graphs(Vs, Es):
    '''
    convert the nodes and edges into networkx graphs
    '''
    Gs = []
    for i in range(len(Vs)):
        V = Vs[i]
        E = Es[i]
        G = nx.Graph()
        # G.add_nodes_from(V.keys())
        for u,nbrs in E.items():
            for v in nbrs:
                G.add_edge(u, v)
        Gs.append(G)
    return Gs

In [20]:
Gs = create_nx_graphs(Vs, Es)

In [21]:
def random_walk(G, u, k, node_labels):
    curr_node = u
    walk = []
    for i in range(k):
        idx = random.randint(0,len(list(G[curr_node]))-1)
        curr_node = list(G[curr_node])[idx]
        walk.append(node_labels[curr_node])
    return tuple(walk)

In [22]:
# generate a number of random walks per node
# such that walks consist of the provided node labels 
nr_walks_per_node = 10
walks_sets = {}
nodes = set()
for G in Gs:
    for u in G.nodes():
        for j in range(nr_walks_per_node):
            walk = random_walk(G, u, 4, node_labels)
            walks_sets.setdefault(walk, set())
            walks_sets[walk].add(u)
            nodes.add(u)
walks = {}
for w, ws in walks_sets.items():
    walks[w] = list(ws)

In [23]:
len(walks)

889

In [24]:
len(walks), len(nodes)

(889, 31175)

In [25]:
class SimilarityGen(tf.keras.utils.Sequence):
    """
        A generator class that will generate positive and negative node pairs.
    """    
    
    def __init__(self, 
                 nodes,
                 walks,
                 nr_neg_samples, 
                 batch_size):
        
        """
        Initialization
        :nodes: the nodes in all graphs
        :walks: a hash map with the nodes per walk
        :param node_to_int: a mapping of node ids to consecutive integers
        :param nr_neg_samples: the number of negative samples for positive pair
        :param batch_size: how many samples to generate
        """
        assert nr_neg_samples >= 1
        self.nodes = nodes
        self.walks = walks # the labeled walks for sampling of positive examples
        # self.node_to_int = node_to_int
        self.nr_neg_samples = nr_neg_samples # by how mach to multiply the number pf positive samples
        self.batch_size = batch_size
        
         
    # how many samples to generate per epoch        
    def __len__(self):
        return 20000
    
    def __getitem__(self, idx):
        i = 0
        samples = []
        labels = []
        while i < self.batch_size:
            i += 1
            walk_idx = np.random.randint(len(self.walks))
            walk = list(self.walks.keys())[walk_idx]
            while len(self.walks[walk]) < 4:
                walk_idx = np.random.randint(len(self.walks))
                walk = list(self.walks.keys())[walk_idx]
            u_idx = np.random.randint(len(self.walks[walk]))
            v_idx = u_idx 
            while v_idx == u_idx:
                v_idx = np.random.randint(len(self.walks[walk]))
            
            neg_idx = np.random.choice(len(nodes), self.nr_neg_samples)
            neg = [self.nodes[idx] for idx in neg_idx]
            
            # generate positive samples from the random walk
            samples.append((self.walks[walk][u_idx], self.walks[walk][v_idx]))
            labels.append(1)
            
            # generate negative samples, a multiple of the walk length
            samples.extend([(self.walks[walk][u_idx], n) for n in neg])
            labels.extend(self.nr_neg_samples*[0])
        return np.array(samples), np.array(labels)

In [26]:
nodes = list(nodes)
gen = SimilarityGen(nodes, walks, nr_neg_samples=2, batch_size=1)

In [27]:
gen[100]

(array([[30080, 10672],
        [30080, 26554],
        [30080, 27547]]),
 array([1, 0, 0]))

In [28]:
class Node2Vec(Model):
    """
    The discriminative model that trains node embeddings. 
    Essentially, this is word2vec with negative sampling.
    """
    def __init__(self, nr_nodes, embedding_dim, *args, **kwargs):
        super(Node2Vec, self).__init__(self, args, kwargs)
        self.nr_nodes = nr_nodes
        self.embedding_dim = embedding_dim
        self.target_embedding = Embedding(nr_nodes,
                                          embedding_dim,
                                          embeddings_initializer="RandomNormal",
                                          input_length=1,
                                          name="node_embedding")
        self.context_embedding = Embedding(nr_nodes,
                                           embedding_dim,
                                           embeddings_initializer="RandomNormal",
                                           input_length=1,
                                           name="context_embedding")
        self.dots = Dot(axes=1)
        self.flatten = Flatten()
        self.dense = Dense(1, activation='sigmoid')

    def call(self, pair):
        target, context = pair[:,0], pair[:,1]
        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)
        dots = self.dots([context_emb, word_emb])
        flat = self.flatten(dots)
        return self.dense(flat)
    
    def summary(self):
        x = Input(shape=(2,))
        model = Model(inputs=[x], outputs=self.call(x))
        return model.summary()

In [29]:
max(nodes)

31385

In [30]:
# this is a small graph, so the embedding dimensionality can be also small 
embedding_dim = 10
nr_nodes = max(nodes)+1
n2v = Node2Vec(nr_nodes=nr_nodes, embedding_dim=embedding_dim)
n2v.build(input_shape=(nr_nodes, embedding_dim))
n2v.compile(optimizer='adam',
                 loss= 'binary_crossentropy', 
                 metrics=['accuracy', 'AUC'])

In [31]:
n2v.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None,)              0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None,)              0           input_1[0][0]                    
__________________________________________________________________________________________________
context_embedding (Embedding)   (None, 10)           313860      tf.__operators__.getitem_1[0][0] 
______________________________________________________________________________________________

In [32]:
n2v.fit(gen, epochs=10)

Epoch 1/10
20000/20000 [==============================] - 84s 4ms/step - loss: 0.5804 - accuracy: 0.7152 - auc: 0.6702
Epoch 2/10
20000/20000 [==============================] - 83s 4ms/step - loss: 0.4552 - accuracy: 0.8111 - auc: 0.7674
Epoch 3/10
20000/20000 [==============================] - 82s 4ms/step - loss: 0.4058 - accuracy: 0.8410 - auc: 0.8120
Epoch 4/10
20000/20000 [==============================] - 84s 4ms/step - loss: 0.3804 - accuracy: 0.8564 - auc: 0.8382
Epoch 5/10
20000/20000 [==============================] - 86s 4ms/step - loss: 0.3606 - accuracy: 0.8648 - auc: 0.8590
Epoch 6/10
20000/20000 [==============================] - 85s 4ms/step - loss: 0.3432 - accuracy: 0.8730 - auc: 0.8797
Epoch 7/10
20000/20000 [==============================] - 85s 4ms/step - loss: 0.3310 - accuracy: 0.8781 - auc: 0.9009
Epoch 8/10
20000/20000 [==============================] - 84s 4ms/step - loss: 0.3230 - accuracy: 0.8797 - auc: 0.9136
Epoch 9/10
20000/20000 [========================

In [36]:
embeddings = n2v.get_layer('node_embedding').get_weights()[0]

In [37]:
node_embeddings = {idx: emb for idx, emb in enumerate(embeddings)}

In [96]:
explicit_map_dim = 100
poly_ts = PolynomialCountSketch(degree=2, gamma=10, random_state=1, n_components=explicit_map_dim)

In [97]:
# create the graph emebddings using explicit feature maps for the polynomial kernel.
X = []
for V in Vs:
    graph_emb = np.array([.0 for _ in range(explicit_map_dim)])
    for v in V:
        emb_v = poly_ts.fit_transform([node_embeddings[v]])
        graph_emb = np.add(graph_emb, emb_v[0])
    graph_emb = graph_emb/len(V)
    X.append(graph_emb)
        

In [98]:
X = np.array(X)

In [99]:
len(X), len(classes)

(2000, 2000)

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.3, random_state=1)

In [101]:
X_train.shape

(1400, 100)

In [102]:
def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)[:,1]
    print("AUC score: ", auc(y_test, y_pred))

In [112]:
model_logreg = LogisticRegression(C=1)
model_rf = RandomForestClassifier(max_depth=12, random_state=0)

In [113]:
# No luck with Logistic Regression
train_and_evaluate(model_logreg, X_train, X_test, y_train, y_test)

AUC score:  0.5689402810304449


In [114]:
# The random forest classifier looks much better
train_and_evaluate(model_rf, X_train, X_test, y_train, y_test)

AUC score:  0.8322965456674473
